In [ ]:
# !py -3 -m pip install pandas
# !py -3 -m pip install teradatasql
# !py -3 -m pip install teradatasqlalchemy

In [ ]:
import pandas as pd
# import teradatasql
import sqlalchemy 

In [ ]:
user_name = 'KHU9683'
pword = 'Mer82@dme'
host_name='EDWPROD.DW.MEDCITY.NET'

In [ ]:
td_engine = sqlalchemy.create_engine('teradatasql://' + host_name + '/?user=' + user_name + '&password=' + pword + '&logmech=LDAP')

In [ ]:
#path of file with table name and database name in csv file 
input_path = "InputFiles\\taleo_tables.csv"
output_folder = "OutputFiles\\InsertSelect\\"

In [ ]:
#Write file to local directory
def write_file_local(path,file_data):
    
    with open(path, 'w') as file:
        file_string = '\n'.join(file_data)
        file.write(file_string)

In [ ]:
#get the DDLs for the tables from teradata 
def gen_insert_select_ext_staging():
    # with teradatasql.connect(host=host_name, user=user_name, password=pword, logmech="LDAP") as connect:
        df = pd.read_csv(input_path, index_col=None)        
        insert_select_list = []
        for index, row in df.iterrows():
            try:
                database_name = str(row['Database']).strip().lower()
                table_name = str(row['Table']).strip().lower()
                source_system_code = str(row['Src_Sys_Code']).strip().upper()
                output_path = output_folder + table_name + ".sql"

                if database_name == "edwhr_staging":
                    database_param = "{{ params.param_hr_stage_dataset_name }}"

                insert_select_list.append("INSERT INTO " + database_param + "." + table_name + "(")

                query = "Select LOWER(columnname) AS columnname, UPPER(columntype) AS columntype "\
                    "From dbc.columnsV WHERE UPPER(DatabaseName) = '{}' AND UPPER(TableName) = '{}'"\
                    ";".format(database_name, table_name)
                results_df = pd.read_sql(query, td_engine)
                for row_no, record in results_df.iterrows():
                    insert_select_list.append(chr(9) + record['columnname'] + ("," if((row_no + 1 ) < len(results_df)) else ""))

                insert_select_list.append(")")
                insert_select_list.append("SELECT")

                for row_no, record in results_df.iterrows():
                    if record['columnname'] in \
                    ('dw_last_update_date_time', 'dw_last_update_date_tim',
                    'sk_generated_date_time', 'dw_last_update_time'):
                        insert_select_list.append(chr(9) + "DATETIME_TRUNC(CURRENT_DATETIME('US/Central'), SECOND)"
                        + ("," if((row_no + 1 ) < len(results_df)) else ""))
                    elif record['columnname'] == 'source_system_code':
                        insert_select_list.append(chr(9) + "'{}'".format(source_system_code)
                        + ("," if((row_no + 1 ) < len(results_df)) else ""))
                    else:
                        insert_select_list.append(chr(9) + record['columnname']
                        + ("," if((row_no + 1 ) < len(results_df)) else ""))

                insert_select_list.append("FROM " + database_param + "." + table_name + "_ext;")

            except Exception as e1:
                print(e1)
                print("Database : {}, Table : {}".format(database_name,table_name))
                pass

        write_file_local(output_path, insert_select_list)
        print("Tables for Record Counts : ", len(df))

In [ ]:
print("Begin of Processing")

gen_insert_select_ext_staging()

print("End of Processing")